In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
import time
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset
import torch

import pickle

In [2]:
import pyro
import pyro.distributions as dist
from pyro.nn import PyroModule, PyroSample

c:\Users\Revalda Putawara\.conda\envs\bnntest\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import pandas as pd

In [4]:
device = torch.device("cuda")

In [5]:
from torchvision.datasets import ImageFolder

In [6]:
dataset = ImageFolder(
    root="data/shipsnet/foldered",
    transform=transforms.ToTensor()
)

In [7]:
loader = DataLoader(
    dataset,
    batch_size=64,
    shuffle=False,
    num_workers=1)

In [8]:
shipsnet_mean = [0.4119, 0.4243, 0.3724]
shipsnet_std = [0.1899, 0.1569, 0.1515]

def load_data(batch_size=16):
    transform = transforms.Compose([
        transforms.Resize((64, 64)),
        transforms.ToTensor(),
        transforms.Normalize(mean=shipsnet_mean, 
                             std=shipsnet_std)
    ])

    dataset = ImageFolder(
    root="data/shipsnet/foldered",
    transform=transform
    )

    torch.manual_seed(42)

    #train_size = int(0.8 * len(dataset))
    #test_size = len(dataset) - train_size
    #train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
    
    with open('datasplit/shipsnet_split_indices.pkl', 'rb') as f:
        split = pickle.load(f)
        train_dataset = Subset(dataset, split['train'])
        test_dataset = Subset(dataset, split['test'])

    # Add num_workers and pin_memory for faster data loading
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, 
                             num_workers=4, pin_memory=True, persistent_workers=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size,
                            num_workers=4, pin_memory=True, persistent_workers=True)
    return train_loader, test_loader, train_dataset, test_dataset

In [9]:
train_loader, test_loader, train_ds, test_ds = load_data(16)

In [10]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import transforms

In [11]:
lr          = 1e-3
num_epochs  = 20

In [12]:
# ─── 5. Model Definition ──────────────────────────────────────────────────────
class ShipsCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1), nn.ReLU(inplace=True), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(inplace=True), nn.MaxPool2d(2),
            nn.Conv2d(64,128, 3, padding=1), nn.ReLU(inplace=True), nn.MaxPool2d(2),
            nn.AdaptiveAvgPool2d((1,1))     # → [B,128,1,1]
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),                   # → [B,128]
            nn.Linear(128, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(256, 2)
        )

    def forward(self, x):
        x = self.features(x)
        return self.classifier(x)

model = ShipsCNN().to(device)

# ─── 6. Loss, Optimizer & Scheduler ───────────────────────────────────────────
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

# ─── 7. Training Loop ─────────────────────────────────────────────────────────
for epoch in range(1, num_epochs + 1):
    # — Train —
    model.train()
    running_loss = running_corrects = 0
    for imgs, labels in train_loader:
        imgs  = imgs.to(device, non_blocking=True)
        labels= labels.to(device, non_blocking=True)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        preds = outputs.argmax(dim=1)
        running_loss    += loss.item() * imgs.size(0)
        running_corrects+= (preds == labels).sum().item()

    epoch_loss = running_loss / len(train_ds)
    epoch_acc  = running_corrects / len(train_ds)

    # — Validate —
    model.eval()
    val_loss = val_corrects = 0
    with torch.no_grad():
        for imgs, labels in test_loader:
            imgs   = imgs.to(device)
            labels = labels.to(device)
            outputs= model(imgs)
            loss   = criterion(outputs, labels)
            preds  = outputs.argmax(dim=1)

            val_loss     += loss.item() * imgs.size(0)
            val_corrects += (preds == labels).sum().item()

    val_loss = val_loss / len(test_ds)
    val_acc  = val_corrects / len(test_ds)
    scheduler.step()

    print(f"Epoch {epoch:2d}/{num_epochs} "
          f"Train: loss={epoch_loss:.4f}, acc={epoch_acc:.4f} | "
          f"Val:   loss={val_loss:.4f}, acc={val_acc:.4f}")

# ─── 8. Save Checkpoint ───────────────────────────────────────────────────────
#os.makedirs("checkpoints", exist_ok=True)
#torch.save(model.state_dict(), "checkpoints/shipsnet_cnn.pth")
#print("Model saved to checkpoints/shipsnet_cnn.pth")

Epoch  1/20 Train: loss=0.5575, acc=0.7488 | Val:   loss=0.4904, acc=0.7500
Epoch  2/20 Train: loss=0.3250, acc=0.8678 | Val:   loss=0.3885, acc=0.8213
Epoch  3/20 Train: loss=0.1995, acc=0.9266 | Val:   loss=0.2180, acc=0.9125
Epoch  4/20 Train: loss=0.1566, acc=0.9403 | Val:   loss=0.1219, acc=0.9525
Epoch  5/20 Train: loss=0.1516, acc=0.9437 | Val:   loss=0.1388, acc=0.9525
Epoch  6/20 Train: loss=0.1344, acc=0.9509 | Val:   loss=0.1263, acc=0.9437
Epoch  7/20 Train: loss=0.1242, acc=0.9575 | Val:   loss=0.1079, acc=0.9575
Epoch  8/20 Train: loss=0.1112, acc=0.9566 | Val:   loss=0.1032, acc=0.9563
Epoch  9/20 Train: loss=0.1011, acc=0.9672 | Val:   loss=0.0829, acc=0.9675
Epoch 10/20 Train: loss=0.0890, acc=0.9675 | Val:   loss=0.1250, acc=0.9500
Epoch 11/20 Train: loss=0.0695, acc=0.9744 | Val:   loss=0.0708, acc=0.9762
Epoch 12/20 Train: loss=0.0603, acc=0.9812 | Val:   loss=0.0670, acc=0.9775
Epoch 13/20 Train: loss=0.0531, acc=0.9812 | Val:   loss=0.0637, acc=0.9762
Epoch 14/20 

In [ ]:
# ─── 6. Save Model ─────────────────────────────────────────────────────────────
os.makedirs("checkpoints", exist_ok=True)
torch.save(model.state_dict(), "checkpoints/shipsnet_cnn.pth")
print("Training complete. Model saved to checkpoints/shipsnet_cnn.pth")